In [1]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import StratifiedKFold
import seaborn as sns
import datetime as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder

%matplotlib inline

In [2]:
import pickle

with open('train_hack.pckl', 'rb') as f:
    train_hack = pickle.load(f)

In [3]:
train_hack.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1748890 entries, 0 to 99930
Data columns (total 17 columns):
can_buy               bool
can_promote           bool
category              int64
contacts_visible      bool
date_created          int64
delivery_available    bool
description           object
fields                object
id                    object
images                object
location              object
mortgage_available    bool
name                  object
payment_available     bool
price                 int64
subcategory           int64
subway                object
dtypes: bool(6), int64(4), object(7)
memory usage: 170.1+ MB


In [ ]:
with open('test_hack.pckl', 'rb') as f:
    test_hack = pickle.load(f)

In [ ]:
test_hack.info()

In [4]:
def preproc_data(df_input):
    df_output = df_input.copy()

 
    df_output = df_output.drop(['images'], axis=1)
    df_output = df_output.drop(['subcategory'], axis=1)
    df_output = df_output.drop(['fields'], axis=1)
    df_output = df_output.drop(['description'], axis=1)#иногда в этом поле какой-то словесный мусор. 
    df_output = df_output.drop(['date_created'], axis=1)
    df_output = df_output.drop(['location'], axis=1)#можно было бы и с локацией попробовать..
    df_output = df_output.drop(['subway'], axis=1)#в трейне только 235 записей с метро
    df_output = df_output.drop(['name'], axis=1)#не успела досчитаться функция forest.fit с векторами до дедлайна... 
    df_output['mortgage_available']=df_output['mortgage_available'].map(lambda x: 1 if True else 0)
    df_output['payment_available']=df_output['payment_available'].map(lambda x: 1 if True else 0)
    df_output['delivery_available']=df_output['delivery_available'].map(lambda x: 1 if True else 0)
    df_output['contacts_visible']=df_output['contacts_visible'].map(lambda x: 1 if True else 0)
    df_output['can_promote']=df_output['can_promote'].map(lambda x: 1 if True else 0)
    df_output['can_buy']=df_output['can_buy'].map(lambda x: 1 if True else 0)
    df_output=df_output.reset_index(drop=True)

    return df_output

Я считала также rlmse и подбирала параметры для RandomForestRegressor, но решила убрать это из ноутбука, т.к. на результат конечный это не влияет

In [5]:
bigtrain = train_hack.pipe(preproc_data)

In [6]:
bigtrain['price']=bigtrain['price']/100

In [7]:
bigtrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1748890 entries, 0 to 1748889
Data columns (total 9 columns):
can_buy               int64
can_promote           int64
category              int64
contacts_visible      int64
delivery_available    int64
id                    object
mortgage_available    int64
payment_available     int64
price                 float64
dtypes: float64(1), int64(7), object(1)
memory usage: 120.1+ MB


In [ ]:
bigtest = test_hack.pipe(preproc_data)

In [8]:
ids = bigtrain['id']
bigtrain = bigtrain.drop(['id'], axis=1)
y = bigtrain['price']
X = bigtrain.drop('price', axis=1)

In [11]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [12]:
from sklearn.ensemble import RandomForestRegressor
forest = Pipeline([
    ('scaler', StandardScaler()),
    ('forest', RandomForestRegressor(n_estimators=100, n_jobs=2))
])

In [13]:
forest.fit(X,y)

Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('forest', RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=2,
           oob_score=False, random_state=None, verbose=0, warm_start=False))])

In [ ]:
idsTest = bigtest['id']
bigtest = bigtest.drop(['id'], axis=1)

In [ ]:
forest_pred = forest.predict(bigtest)

In [ ]:
submit = pd.DataFrame({'id':idsTest,'price':forest_pred})

In [ ]:
submit.to_csv('submission1.csv', sep=',', index=False)